In [1]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/analysis_lib.ipynb"

In [2]:
args = getargs()

/home/ibis/gregor.sturm/bin/anaconda/lib/python2.7/site-packages/nbwrapper.py:31: RuntimeWarning: no arguments passed!
  warnings.warn("no arguments passed!", RuntimeWarning)


In [3]:
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params_3mer.pickle",
    "ncores": 4,
    "nmers": 3,
    "multivariate": True
}

# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params_{0}mer.pickle".format(NMERS),
#     "ncores": 62,
#     "nmers": 3,
#     "multivariate": True
# }

In [4]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [5]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = OrderedDict(HMM_PARAMS)
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS.keys() == ALL_KMERS

# Train Model 

In [6]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [7]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [8]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (20/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (29/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [9]:
mk_transmat = mk_transmat2

In [10]:
F = ghmm.Float()  # emission domain of this model

def mk_model_multivariate():
    # example code for a continuous HMM with gaussian emissions

    A = mk_transmat(NMERS)
    B = [ [ 
            [float(df[['mean']].mean()), float(df[['stdv']].mean())], #mu1, mu2
            [x for x in chain(*df[['mean', 'stdv']].cov().values.tolist())], #covariance matrix
        ] for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.MultivariateGaussianDistribution(F), A, B, pi)
    return model

In [11]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = [ [float(df[['mean']].mean()), float(df[['mean']].std())] #mu1, stdv
            for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [12]:
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [13]:
model = mk_model()
s = str(model)
print(s)


HMM Overview:
Number of states: 64
maximum Number of mixture components: 1
Number of dimensions: 2

State number 0:
Initial probability: 0.015625
Number of mixture components: 1

  Emission number 0:
    emission type: 6
    emission weight: 1.0
    mean: [70.16634359689922, 1.4638138587468255]
    covariance matrix: [14.74402884794052, 0.34259824972145786, 0.34259824972145786, 0.1345697766710737]
    inverse of covariance matrix: [0.07208861758745962, -0.1835288340462259, -0.1835288340462259, 7.898331138020878]
    determinant of covariance matrix: 1.86672710859
    cholesky decomposition of covariance matrix: [14.74402884794052, 0.34259824972145786, 0.34259824972145786, 0.1345697766710737]
    fix: 0

  Class : 0
    Outgoing transitions:
      transition to state 0 with probability = 0.025
      transition to state 1 with probability = 0.025
      transition to state 2 with probability = 0.025
      transition to state 3 with probability = 0.025
      transition to state 4 with pro

In [14]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [15]:
def predict(mixed):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    if MULTIVARIATE: 
        emissions = [x for x in chain(*mixed)]    
    else: 
        emissions = [x[0] for x in mixed]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [16]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [17]:
predict(seq)

'ACGAGCTATC'

# Validate Model 

In [18]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [19]:
assert os.path.isfile(args["events"])

In [20]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>mmusMT_PCR1']
GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACACAAAGGTTTGGTCCTGGCCTTATAATTAATTA


In [21]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [22]:
prepare_filemap(file_data)

In [23]:
def train_read(file_obj):    
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    seq = ghmm.EmissionSequence(F, events)
    model.baumWelch(seq)
    

In [24]:
def basecall_read(file_obj):
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [25]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [26]:
p = Pool(args["ncores"])

In [27]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, file_data), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

Prediction: 
done 100.000000%

### Stats

In [28]:
types = ["metrichor", "called", "random"]
fasta_files = {t: "{0}.{1}.fa".format(args["out_basename"], t) for t in types}

In [29]:
## metrichor fasta
with open(fasta_files["metrichor"], 'w') as f: 
    for file_obj in file_data: 
        f.write(">ch{0}_file{1}_metrichor".format(file_obj["channel"], file_obj["file_id"])+ "\n")
        f.write(file_obj["fastq"].split("\n")[1] + "\n")

In [30]:
## called fasta/random fasta
with open(fasta_files["called"], 'w') as f: 
    with open(fasta_files["random"], 'w') as fr:
        for channel, file_id, seq in results: 
            f.write(">ch{0}_file{1}_called".format(file_obj["channel"], file_obj["file_id"])+ "\n")
            fr.write(">ch{0}_file{1}_random".format(file_obj["channel"], file_obj["file_id"])+ "\n")
            f.write(seq + "\n")
            fr.write("".join([random.choice("ACGT") for _ in range(len(seq))]))

In [31]:
for t in types: 
    sam_file = "{0}.{1}.sam".format(args["out_basename"], t)
    graphmap(ref_file, fasta_files[t], sam_file, args["ncores"])
    prepare_sam("{0}.{1}".format(args["out_basename"], t))

[Index 09:02:56] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 09:02:56] Index already exists. Loading from file.
[Index 09:02:56] Secondary index already exists. Loading from file.
[Index 09:02:56] Index loaded in 0.53 sec.
[Index 09:02:56] Memory consumption: [currentRSS = 515 MB, peakRSS = 515 MB]

[Run 09:02:56] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 09:02:56] Reference genome is assumed to be linear.
[Run 09:02:56] Only one alignment will be reported per mapped read.
[ProcessReads 09:02:56] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 09:02:56] Batch of 81 reads (0 MiB) loaded in 0.01 sec. (29963432 bases)
[ProcessReads 09:02:56] Memory consumption: [currentRSS = 515 MB, peakRSS = 515 MB]
[ProcessReads 09:02:56] Using 4 threads.
[ProcessReads 09:02:58] [CPU time: 4.96 sec, RSS: 531 MB] Read: 81/81 (100.00%) [m: 80, u: 1]                     

In [32]:
def mk_stat(t):
    samfile = pysam.AlignmentFile("{0}.{1}.sorted.bam".format(args["out_basename"], t))
    sst = samstats(samfile, ref)
    return pandas.DataFrame(sst.print_summary())

In [33]:
p = Pool(args["ncores"])

In [34]:
try:
    stats = p.map(mk_stat, types)
    p.close()
except KeyboardInterrupt:
    p.terminate()

In [35]:
print(types)
side_by_side(*stats)

['metrichor', 'called', 'random']


,0,1,2,3
0,mapped_reads/total_reads,80,81,98.765432%
1,significant_reads/total_reads,68,81,83.950617%
2,mapped_nts/total_nts,387166,454059,85.267774%
3,editdistance/alignment_length,201132,442591,45.444214%
4,alignment_score/alignment_length,319854,442591,72.268528%
5,SNPs/mapped_nts,78814,387166,20.356643%
6,ins/mapped_nts,66821,387166,17.259005%
7,del/mapped_nts,55425,387166,14.315565%
,0,1,2,3
0,mapped_reads/total_reads,72,81,88.888889%


In [36]:
for t, df in zip(types, stats):
    with open("{0}.stats.{1}.html".format(args["out_basename"], t), 'w') as f:
        f.write(df.to_html())

In [37]:
# def score_consensus(t):
#     consensus = mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], t), ref_file)
#     return(consensus)
#     consensus = consensus.split("\n")[1].to_upper()
#     score = needle(ref, consensus)
#     return (consensus, score)

In [38]:
# p = Pool(args["ncores"])
# try:
#     consensus = p.map(score_consensus, types)
#     p.close()
# except KeyboardInterrupt:
#     p.terminate()

In [39]:
# consensus

In [40]:
# mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], "metrichor"), ref_file)